# Deformable (nonlinear) registration

This notebook shows example of nonlinear, deformable registration with elastix. It demonstrates some
common pitfalls, and how to address them.

One of the main challenges is controlling the amount of deformation produced by the algorithm. We will learn
how to adjust algorithm parameters to balance accuracy without introducing too much distortion.

Finally, we will briefly learn about one way to quantitatively measure distortion using the "jacobian determinant."

As before, we start with importing libraries, defining some functions, and loading the example images.

In [ ]:
import os
from itertools import chain

import numpy as np
import SimpleITK as sitk

import itkwidgets
from itkwidgets import view

def itk_image(np_array, resolution):
    img = sitk.GetImageFromArray(np_array.astype(np.float32))
    img.SetOrigin((0, 0, 0))
    img.SetSpacing(resolution)
    return img

def imwrite(image, filename):
    writer = sitk.ImageFileWriter()
    writer.SetFileName(filename)
    writer.Execute(image)
    
def run_elastix(fixed_image, moving_image, params):
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixed_image)
    elastixImageFilter.SetMovingImage(moving_image)
    elastixImageFilter.SetParameterMap(params)
    elastixImageFilter.Execute()
    return elastixImageFilter

def print_parameters(elastix_parameters):
    for k,v in elastix_parameters.iteritems():
        print(f'{k} \n\t {v}')


In [ ]:
fixed_image_path='../sampleImages/jrc18_down.nrrd'
fixed_image = sitk.ReadImage(fixed_image_path)

moving_image_path = '../sampleImages/jrc10_down.nrrd'
moving_image = sitk.ReadImage(moving_image_path)

First, we'll load some default parameters from a file, run elastix with those parameters, and visualize the result.

In [ ]:
default_params = sitk.ReadParameterFile('../elastixParameters/DefaultBsplineParams.txt')
elastix_nonlinear = run_elastix(fixed_image, moving_image, default_params ) 
result_image = elastix_nonlinear.GetResultImage()
sitk.WriteImage(result_image, '../sampleImages/defaultbspline_result_img.nrrd')

It takes quite awhile for the above to run, and the result is pretty bad.  It may look okay when zoomed out, but zooming in and scrolling in z will show that there is very high deformation.

If it's difficult to appreciate the high deformation below, try opening the file in Fiji.  It will be located here: `sampleImages/defaultbspline_result_img.nrrd`.

In [ ]:
view(result_image)

## Let's make it better

One very common approach to registration is to break it into multiple registration steps, that start simple and become more complex.

Next we will first run an affine registration. It's simple and so will not produce distortion.  After that, we will **run our nonlinear registration using the result of the affine registration**.  

In [ ]:
affine_params = sitk.ReadParameterFile('../elastixParameters/DefaultSmoother_Affine.txt')
elastixAffine = run_elastix(fixed_image, moving_image, affine_params ) 
sitk.WriteImage(elastixAffine.GetResultImage(), 'affine_result_img.nrrd')

In [ ]:
# use the affine as the new moving image
affine_result = elastixAffine.GetResultImage()
bspline_params = sitk.ReadParameterFile('../elastixParameters/DefaultSmoother_Bspline.txt')
elastixAffineBspline = run_elastix(fixed_image, affine_result, bspline_params ) 
sitk.WriteImage(elastixAffineBspline.GetResultImage(), '../sampleImages/affine_bspline_result_img.nrrd')

Open the resulting image at `sampleImages/affine_bspline_result_img.nrrd` and compare to the previous result. It looks much better (less deformed). There is one more improvement to do.

## There's still a little problem...

In the two step registration we just did, running elastix the first time created an intermediate transformed image that we transformed a second time on the second run through elastix. We ran the image transformation routine twice. Whenever we transform an image, we need to interpolate that image (remember the video lecture). So effectively, **we interpolated our moving image twice.**

**Interpolating an image blurs the image.** This is demonstrated in this example ([credit Robert Hasse](https://twitter.com/haesleinhuepf/status/1088546103866388481))

<img src="haesleinhuepf-1088546103866388481_7s.gif">

For the movie on the right, the image is repeatedly interpolated as it is rotated more and more. The repeated interpolation causes the blurring effect. **We can remove this undesirable blurring by only interpolating once.**

For the movie on the right, to generate each subsequent frame, we concatenate the transformations, always start with the original image, and iterpolate it once. Since only one interpolation is every performed, the amount of blurring does not increase.

## Fix it

Fortunately, we can fix this problem by telling elastix about both registration steps, letting it handle concatenating the transformations. As a result, the image we get will only be interpolated once, and have less blurring.

To do that, we make a sequence (vector) of elastix parameters like this:

In [ ]:
parameter_sequence = sitk.VectorOfParameterMap()

# The first (affine) set of parameters
parameter_sequence.append(affine_params)

# The second (nonlinear) set of parameters
parameter_sequence.append(bspline_params)

# run elastix with the parameter vector
elastix_sequence = run_elastix(fixed_image, moving_image, parameter_sequence ) 
sitk.WriteImage(elastix_sequence.GetResultImage(), '../sampleImages/affinebspline_result_img.nrrd')

If you look through the output, you will see that it runs both the affine and nonlinear steps.

In [ ]:
transform_sequence = elastix_sequence.GetTransformParameterMap()

transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(transform_sequence)
transformixImageFilter.SetMovingImage(moving_image)
transformixImageFilter.Execute()
sitk.WriteImage(transformixImageFilter.GetResultImage(), '../sampleImages/tform_res.nrrd')

Just like in `similarity_metrics.ipynb`, we can get the transform parameters.  But now, we have a list of transformations.  Notice `ParameterMap 0` and `Parameter Map 1` below that refer to the affine and nonlinear transforms, respectively.

Also like before, we can still change the desired output image size and and resolution as we say in `multi_resolution.ipynb`. We'll do that and run transformix.  If you would like, try running transformix on the high-resolution moving image as well.

In [ ]:
transform_sequence[0]['Size'] = ['207', '100', '60']
transform_sequence[0]['Spacing'] = ['3.0', '3.0', '3.0']
transform_sequence[1]['Size'] = ['207', '100', '60']
transform_sequence[1]['Spacing'] = ['3.0', '3.0', '3.0']
sitk.PrintParameterMap( transform_sequence )

In [ ]:
transformixImageFilter = sitk.TransformixImageFilter()
transformixImageFilter.SetTransformParameterMap(transform_sequence)
transformixImageFilter.SetMovingImage(moving_image)
transformixImageFilter.Execute()

## Bonus: quantifying deformation

The [jacobian determinant](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant) measures how much a transformation locally expands or shrinks space. It gives a measurement at every point in space, the jacobian determinant result is itself an image.

If the image contains areas of expansion (big positive values) and areas of shrinking (values close to zero), then it is a sign that there is lots of distortion, especially if the values change quickly over space.

It is possible for the jacobian determinant to be negative, and if so it is a very bad sign. It indicates that the transformation is not invertible - that multiple points in the moving space end up at the same output location.  In other words that space "has folded over itself".  **Negative values of the jacobian determinant are a big problem** 

The function below shows how to compute the jacobian determinant with transformix:

In [ ]:
def jacobian_determinant(params, output_directory='../sampleImages'):
    """
    Computes an image showing the jacobian determinant and
    writes it to the output_directory.
    
    returns the transformix filter
    """
    transformixImageFilter = sitk.TransformixImageFilter()
    transformixImageFilter.SetTransformParameterMap(params)
    transformixImageFilter.SetMovingImage(moving_image)
    transformixImageFilter.SetOutputDirectory(output_directory)
    transformixImageFilter.SetComputeDeterminantOfSpatialJacobian(True)
    transformixImageFilter.Execute()
    return transformixImageFilter
    

Now let's use that function to compute the jacobian determinant for the first result that we got that qualitatively looked to have lots of distortion.

Then we'll also compute the jacobian determinant for the multi-step registration result that looks smoother.

In [ ]:
jacobian_determinant(elastix_nonlinear.GetTransformParameterMap())
os.rename('../sampleImages/spatialJacobian.nrrd', '../sampleImages/spatialJacobian_nonlinear.nrrd')

jacobian_determinant(elastix_sequence.GetTransformParameterMap())
os.rename('../sampleImages/spatialJacobian.nrrd', '../sampleImages/spatialJacobian_sequence.nrrd')

We expect that the quantitative results agree our qualitative impression.  Let's see if that's true by displaying the histograms.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

jacdet_onestep = sitk.ReadImage('../sampleImages/spatialJacobian_nonlinear.nrrd')
jacdet_twostep = sitk.ReadImage('../sampleImages/spatialJacobian_sequence.nrrd')

ax = plt.hist( sitk.GetArrayFromImage(jacdet_onestep).flatten(), bins=64, label='one step')
plt.gca().hist( sitk.GetArrayFromImage(jacdet_twostep).flatten(), bins=64, label='two step')
plt.legend();

It's clear that the two-step procedure produces values that are more tightly distributed. This indicates that it is much less distorted than our initial one-step run.  

Even worse, the one-step run produces negative values of the jacobian determinant - a very bad sign!

If you're interested, open both images and compare for yourself. They are located at:

* `sampleImages/spatialJacobian_nonlinear.nrrd`
* `sampleImages/spatialJacobian_sequence.nrrd`

## Bonus: displacement fields

The most common way to represent nonlinear transformation is with a displacement field.  By default, elastix does not produce displacement fields, but it is possible to use transformix to generate and save a displacement field.

The example below shows how to do that:

In [ ]:
def displacement_field(params):
    """
    Computes an image showing the jacobian determinant and
    writes it to the output_directory
    
    returns the transformix filter
    """
    print('new dfield')
    transformixImageFilter = sitk.TransformixImageFilter()
    transformixImageFilter.SetTransformParameterMap(params)
    transformixImageFilter.SetMovingImage(moving_image)
    transformixImageFilter.ComputeDeformationFieldOn()
    transformixImageFilter.SetComputeDeformationField(True)
    transformixImageFilter.Execute()
    return transformixImageFilter

In [ ]:
transformix_dfield = displacement_field(elastix_sequence.GetTransformParameterMap())
sitk.WriteImage(transformix_dfield.GetDeformationField(), '../sampleImages/displacement_field.nrrd')